# 1. Prepare datasets

## 1.1 Trees (TNT) and consensus trees are taken from /mnt/work/phylobench/2020-Pfam33 and placed to /home/ruslan_gumerov/genitor_new/Trees/ (folders /eukaryota/ and /prokaryota/)

In [1]:
import os
import random
import numpy as np
import pandas as pd
from Bio import AlignIO, SeqIO

In [2]:
#path to files
PROKARYOTA_TREES_PATH = "/home/ruslan_gumerov/genitor_new/Trees/prokaryota/"
EUKARYOTA_TREES_PATH = "/home/ruslan_gumerov/genitor_new/Trees/eukaryota/"

In [3]:
#seed
random.seed(9091)

## 1.2 Print list of analyzed folders 

In [4]:
prokaryota = [folder for folder in os.listdir(PROKARYOTA_TREES_PATH)]
eukaryota = [folder for folder in os.listdir(EUKARYOTA_TREES_PATH)]
print("Analyzed prokaryota:", ", ".join(prokaryota), "\n")
print("Analyzed eukaryota:", ", ".join(eukaryota))

Analyzed prokaryota: Proteobacteria-60-family, Archaea-60-genus, OtherBacteria-60, Proteobacteria-60-balanced, Firmicutes-60-genus, Actinobacteria-60-genus, Archaea-60-species 

Analyzed eukaryota: Ascomycota-60-genus, Eukaryota-60-class, Chordata-60-family, Fungi-60-family, Protista-60-species, Metazoa-60-balanced, Eukaryota-60-balanced, Fungi-60-balanced, Streptophyta-60-genus


## 1.3 Copy alignment and consensus files to directories

In [ ]:
PATH_TO_INITIAL_DIR = "/mnt/work/phylobench/2020-Pfam33/"

for file in os.listdir(PROKARYOTA_TREES_PATH):
    os.system(f"mkdir {PROKARYOTA_TREES_PATH}{file}/Alignments/")
    os.system(f"cp {PATH_TO_INITIAL_DIR}{file}/Alignments/* {PROKARYOTA_TREES_PATH}{file}/Alignments/")
    
for file in os.listdir(EUKARYOTA_TREES_PATH):
    os.system(f"mkdir {EUKARYOTA_TREES_PATH}{file}/Alignments/")
    os.system(f"cp {PATH_TO_INITIAL_DIR}{file}/Alignments/* {EUKARYOTA_TREES_PATH}{file}/Alignments/")

for folder in os.listdir(PROKARYOTA_TREES_PATH):
    os.system(f"cp {PATH_TO_INITIAL_DIR}{folder}/ncbi-consense.tre {PROKARYOTA_TREES_PATH}{folder}/consensus/")
    
for folder in os.listdir(EUKARYOTA_TREES_PATH):
    os.system(f"cp {PATH_TO_INITIAL_DIR}{folder}/ncbi-consense.tre {EUKARYOTA_TREES_PATH}{folder}/consensus/")

mkdir: cannot create directory ‘/home/ruslan_gumerov/genitor_new/Trees/prokaryota/Proteobacteria-60-family/Alignments/’: File exists
cp: -r not specified; omitting directory '/mnt/work/phylobench/2020-Pfam33/Proteobacteria-60-family/Alignments/Noised1'
cp: -r not specified; omitting directory '/mnt/work/phylobench/2020-Pfam33/Proteobacteria-60-family/Alignments/Noised2'
cp: -r not specified; omitting directory '/mnt/work/phylobench/2020-Pfam33/Proteobacteria-60-family/Alignments/Noised3'
cp: -r not specified; omitting directory '/mnt/work/phylobench/2020-Pfam33/Proteobacteria-60-family/Alignments/Noised4'
cp: -r not specified; omitting directory '/mnt/work/phylobench/2020-Pfam33/Proteobacteria-60-family/Alignments/Noised5'
cp: -r not specified; omitting directory '/mnt/work/phylobench/2020-Pfam33/Proteobacteria-60-family/Alignments/reduced'
mkdir: cannot create directory ‘/home/ruslan_gumerov/genitor_new/Trees/prokaryota/Archaea-60-genus/Alignments/’: File exists
cp: -r not specified; 

cp: -r not specified; omitting directory '/mnt/work/phylobench/2020-Pfam33/Protista-60-species/Alignments/reduced'
cp: -r not specified; omitting directory '/mnt/work/phylobench/2020-Pfam33/Metazoa-60-balanced/Alignments/reduced'
cp: -r not specified; omitting directory '/mnt/work/phylobench/2020-Pfam33/Eukaryota-60-balanced/Alignments/Noised1'
cp: -r not specified; omitting directory '/mnt/work/phylobench/2020-Pfam33/Eukaryota-60-balanced/Alignments/Noised2'
cp: -r not specified; omitting directory '/mnt/work/phylobench/2020-Pfam33/Eukaryota-60-balanced/Alignments/Noised3'
cp: -r not specified; omitting directory '/mnt/work/phylobench/2020-Pfam33/Eukaryota-60-balanced/Alignments/Noised4'
cp: -r not specified; omitting directory '/mnt/work/phylobench/2020-Pfam33/Eukaryota-60-balanced/Alignments/Noised5'
cp: -r not specified; omitting directory '/mnt/work/phylobench/2020-Pfam33/Eukaryota-60-balanced/Alignments/reduced'
cp: -r not specified; omitting directory '/mnt/work/phylobench/2020-

# 2. Prepare PF lists

## 2.1 Create list of PF's for eukaryota and prokaryota

In [8]:
#create empty lists
prokaryota_PF_list = []
eukaryota_PF_list = []

#add PF's to both lists
for folder in os.listdir(PROKARYOTA_TREES_PATH):
    for file in os.listdir(f"{PROKARYOTA_TREES_PATH}{folder}/trees/"):
        prokaryota_PF_list.append(file.split("_")[1])
        
for folder in os.listdir(EUKARYOTA_TREES_PATH):
    for file in os.listdir(f"{EUKARYOTA_TREES_PATH}{folder}/trees/"):
        eukaryota_PF_list.append(file.split("_")[1])

#each PF one time only in sample's subset
prokaryota_PF_list = list(set(prokaryota_PF_list))
eukaryota_PF_list = list(set(eukaryota_PF_list))

print(f"Total amount of unique PF in prokaryota: {len(prokaryota_PF_list)}")
print(f"Total amount of unique PF in eukaryota: {len(eukaryota_PF_list)}")

Total amount of unique PF in prokaryota: 1100
Total amount of unique PF in eukaryota: 4746


## 2.2 Split PF ID's equally or almost equally on 3 groups (train, test, val) and check that they are not intersected between each other

In [9]:
#create list for split
#0 – train, 1 – test, 2 – val
eukaryota_split_list = [0 for x in range(1582)] + [1 for x in range(1582)] + [2 for x in range(1582)]
prokaryota_split_list = [0 for x in range(366)] + [1 for x in range(367)] + [2 for x in range(367)]

#shuffle list randomly
random.shuffle(eukaryota_split_list)
random.shuffle(prokaryota_split_list)

#create nested lists where PF ID's for train, test and val stored
prokaryota_split_nested_list = [[], [], []]
eukaryota_split_nested_list = [[], [], []]

for i, j in zip(prokaryota_PF_list, prokaryota_split_list):
    if j == 0:
        prokaryota_split_nested_list[0].append(i)
    if j == 1:
        prokaryota_split_nested_list[1].append(i)
    if j == 2:
        prokaryota_split_nested_list[2].append(i)

for i, j in zip(eukaryota_PF_list, eukaryota_split_list):
    if j == 0:
        eukaryota_split_nested_list[0].append(i)
    if j == 1:
        eukaryota_split_nested_list[1].append(i)
    if j == 2:
        eukaryota_split_nested_list[2].append(i)
        
print(f"Intersection between eukaryota train and test: {set(eukaryota_split_nested_list[0]).intersection(set(eukaryota_split_nested_list[1]))}")
print(f"Intersection between eukaryota train and val: {set(eukaryota_split_nested_list[0]).intersection(set(eukaryota_split_nested_list[2]))}")
print(f"Intersection between eukaryota test and val: {set(eukaryota_split_nested_list[1]).intersection(set(eukaryota_split_nested_list[2]))}")

print(f"Intersection between prokaryota train and test: {set(prokaryota_split_nested_list[0]).intersection(set(prokaryota_split_nested_list[1]))}")
print(f"Intersection between prokaryota train and val: {set(prokaryota_split_nested_list[0]).intersection(set(prokaryota_split_nested_list[2]))}")
print(f"Intersection between prokaryota test and val: {set(prokaryota_split_nested_list[1]).intersection(set(prokaryota_split_nested_list[2]))}")


Intersection between eukaryota train and test: set()
Intersection between eukaryota train and val: set()
Intersection between eukaryota test and val: set()
Intersection between prokaryota train and test: set()
Intersection between prokaryota train and val: set()
Intersection between prokaryota test and val: set()


# 3. Generate alignment folders

## 3.1 Script for generation of samples of any defined length and amount (prokaryota)

In [154]:
#set up parameters
n_train = 1000
len_train = 12
n_test = 300
len_test = 30
n_val = 300
len_val = 20
OUTPUT_FOLDER_PATH = "/home/ruslan_gumerov/genitor_new/final_subsets/prokaryota/"
train_path = f"{OUTPUT_FOLDER_PATH}/train/"
test_path = f"{OUTPUT_FOLDER_PATH}/test/"
val_path = f"{OUTPUT_FOLDER_PATH}/val/"

In [155]:
#create dataframe with alignments and PF's
taxon_list = []
alignment_list = []
PF_list = []
for taxon in os.listdir(PROKARYOTA_TREES_PATH):
    for alignment in os.listdir(f"{PROKARYOTA_TREES_PATH}{taxon}/Alignments/"):
        PF = alignment.split("_")[1]
        taxon_list.append(taxon)
        alignment_list.append(alignment)
        PF_list.append(PF)
alignments_df = pd.DataFrame({"taxon" : taxon_list, "alignment" : alignment_list, "PF" : PF_list})
print(f"Alignments dataframe created: \n {alignments_df.head()}\n")
print(f"Taxons: {', '.join(set(alignments_df['taxon']))}\n")
print(f"Number of PF's: {len(set(alignments_df['PF']))}\n")

Alignments dataframe created: 
                       taxon         alignment       PF
0  Proteobacteria-60-family  PF_PF00156_1.afa  PF00156
1  Proteobacteria-60-family  PF_PF14849_1.afa  PF14849
2  Proteobacteria-60-family  PF_PF00384_1.afa  PF00384
3  Proteobacteria-60-family  PF_PF01202_1.afa  PF01202
4  Proteobacteria-60-family  PF_PF00004_2.afa  PF00004

Taxons: Archaea-60-species, Archaea-60-genus, Firmicutes-60-genus, OtherBacteria-60, Actinobacteria-60-genus, Proteobacteria-60-family, Proteobacteria-60-balanced

Number of PF's: 1105



In [156]:
#create folders with train, test and val alignments
#create empty folders
random.seed(9091)
os.system(train_path)
os.system(test_path)
os.system(val_path)

#train
_ = 0
while _ < n_train:
    #select random PF from train subset
    PF_selected = random.choice(prokaryota_split_nested_list[0])
    #select random .afa for this PF
    alignment_selected = random.choice(list(alignments_df[alignments_df["PF"] == PF_selected]["alignment"]))
    taxon_selected = alignments_df[alignments_df["alignment"] == alignment_selected][["taxon"]].values[0][0]
    #read alignment, subset randomly specified amount of sequences and write them
    align = AlignIO.read(f"{PROKARYOTA_TREES_PATH}{taxon_selected}/Alignments/{alignment_selected}", "fasta")
    selected_numbers = random.sample(range(0, len(align)), len_train)
    selected_alignments = [align[x] for x in selected_numbers]
    SeqIO.write(selected_alignments, f"{OUTPUT_FOLDER_PATH}train/{PF_selected}_{taxon_selected}_{len_train}_{_}.fasta", "fasta")
    _+=1
    
#test
_ = 0
while _ < n_test:
    #select random PF from test subset
    PF_selected = random.choice(prokaryota_split_nested_list[0])
    #select random .afa for this PF
    alignment_selected = random.choice(list(alignments_df[alignments_df["PF"] == PF_selected]["alignment"]))
    taxon_selected = alignments_df[alignments_df["alignment"] == alignment_selected][["taxon"]].values[0][0]
    #read alignment, subset randomly specified amount of sequences and write them
    align = AlignIO.read(f"{PROKARYOTA_TREES_PATH}{taxon_selected}/Alignments/{alignment_selected}", "fasta")
    selected_numbers = random.sample(range(0, len(align)), len_test)
    selected_alignments = [align[x] for x in selected_numbers]
    SeqIO.write(selected_alignments, f"{OUTPUT_FOLDER_PATH}test/{PF_selected}_{taxon_selected}_{len_test}_{_}.fasta", "fasta")
    _+=1

#val
_ = 0
while _ < n_val:
    #select random PF from val subset
    PF_selected = random.choice(prokaryota_split_nested_list[0])
    #select random .afa for this PF
    alignment_selected = random.choice(list(alignments_df[alignments_df["PF"] == PF_selected]["alignment"]))
    taxon_selected = alignments_df[alignments_df["alignment"] == alignment_selected][["taxon"]].values[0][0]
    #read alignment, subset randomly specified amount of sequences and write them
    align = AlignIO.read(f"{PROKARYOTA_TREES_PATH}{taxon_selected}/Alignments/{alignment_selected}", "fasta")
    selected_numbers = random.sample(range(0, len(align)), len_val)
    selected_alignments = [align[x] for x in selected_numbers]
    SeqIO.write(selected_alignments, f"{OUTPUT_FOLDER_PATH}val/{PF_selected}_{taxon_selected}_{len_val}_{_}.fasta", "fasta")
    _+=1



sh: 1: /home/ruslan_gumerov/genitor_new/final_subsets/prokaryota//train/: Permission denied
sh: 1: /home/ruslan_gumerov/genitor_new/final_subsets/prokaryota//test/: Permission denied
sh: 1: /home/ruslan_gumerov/genitor_new/final_subsets/prokaryota//val/: Permission denied


In [15]:
#Validation

print(f"Amount of alignments in train: {len(os.listdir(train_path))}")
print("Number of sequences in the train alignment:", os.popen(f"grep -o '>' {train_path}{os.listdir(train_path)[0]} | wc -l").read())

print(f"Amount of alignments in test: {len(os.listdir(test_path))}")
print("Number of sequences in the train alignment:", os.popen(f"grep -o '>' {test_path}{os.listdir(test_path)[0]} | wc -l").read())

print(f"Amount of alignments in val: {len(os.listdir(val_path))}")
print("Number of sequences in the train alignment:", os.popen(f"grep -o '>' {val_path}{os.listdir(val_path)[0]} | wc -l").read())

Amount of alignments in train: 1000
Number of sequences in the train alignment: 12

Amount of alignments in test: 300
Number of sequences in the train alignment: 30

Amount of alignments in val: 300
Number of sequences in the train alignment: 20



## Results stored in /home/ruslan_gumerov/genitor_new/final_subsets/prokaryota/

## 3.2 Script for generation of samples of any defined length and amount (repeat the same for eukaryota)

In [10]:
#set up parameters
n_train = 1000
len_train = 12
n_test = 300
len_test = 30
n_val = 300
len_val = 20
OUTPUT_FOLDER_PATH = "/home/ruslan_gumerov/genitor_new/final_subsets/eukaryota/"
train_path = f"{OUTPUT_FOLDER_PATH}/train/"
test_path = f"{OUTPUT_FOLDER_PATH}/test/"
val_path = f"{OUTPUT_FOLDER_PATH}/val/"

In [11]:
#create dataframe with alignments and PF's
taxon_list = []
alignment_list = []
PF_list = []
for taxon in os.listdir(EUKARYOTA_TREES_PATH):
    for alignment in os.listdir(f"{EUKARYOTA_TREES_PATHEUKARYOTA_PATH}{taxon}/Alignments/"):
        PF = alignment.split("_")[1]
        taxon_list.append(taxon)
        alignment_list.append(alignment)
        PF_list.append(PF)
alignments_df = pd.DataFrame({"taxon" : taxon_list, "alignment" : alignment_list, "PF" : PF_list})
print(f"Alignments dataframe created: \n {alignments_df.head()}\n")
print(f"Taxons: {', '.join(set(alignments_df['taxon']))}\n")
print(f"Number of PF's: {len(set(alignments_df['PF']))}\n")

Alignments dataframe created: 
                  taxon         alignment       PF
0  Ascomycota-60-genus  AS_PF03834_1.afa  PF03834
1  Ascomycota-60-genus  AS_PF05470_1.afa  PF05470
2  Ascomycota-60-genus  AS_PF03571_1.afa  PF03571
3  Ascomycota-60-genus  AS_PF01424_1.afa  PF01424
4  Ascomycota-60-genus  AS_PF05277_1.afa  PF05277

Taxons: Metazoa-60-balanced, Eukaryota-60-class, Fungi-60-balanced, Streptophyta-60-genus, Protista-60-species, Fungi-60-family, Eukaryota-60-balanced, Ascomycota-60-genus, Chordata-60-family

Number of PF's: 4998



In [13]:
#create folders with train, test and val alignments
#create empty folders
random.seed(9091)
os.system(train_path)
os.system(test_path)
os.system(val_path)

#train
_ = 0
while _ < n_train:
    #select random PF from train subset
    PF_selected = random.choice(eukaryota_split_nested_list[0])
    #select random .afa for this PF
    alignment_selected = random.choice(list(alignments_df[alignments_df["PF"] == PF_selected]["alignment"]))
    taxon_selected = alignments_df[alignments_df["alignment"] == alignment_selected][["taxon"]].values[0][0]
    #read alignment, subset randomly specified amount of sequences and write them
    align = AlignIO.read(f"{EUKARYOTA_TREES_PATHEUKARYOTA_PATH}{taxon_selected}/Alignments/{alignment_selected}", "fasta")
    selected_numbers = random.sample(range(0, len(align)), len_train)
    selected_alignments = [align[x] for x in selected_numbers]
    SeqIO.write(selected_alignments, f"{OUTPUT_FOLDER_PATH}train/{PF_selected}_{taxon_selected}_{len_train}_{_}.fasta", "fasta")
    _+=1
    
#test
_ = 0
while _ < n_test:
    #select random PF from test subset
    PF_selected = random.choice(eukaryota_split_nested_list[0])
    #select random .afa for this PF
    alignment_selected = random.choice(list(alignments_df[alignments_df["PF"] == PF_selected]["alignment"]))
    taxon_selected = alignments_df[alignments_df["alignment"] == alignment_selected][["taxon"]].values[0][0]
    #read alignment, subset randomly specified amount of sequences and write them
    align = AlignIO.read(f"{EUKARYOTA_TREES_PATH}{taxon_selected}/Alignments/{alignment_selected}", "fasta")
    selected_numbers = random.sample(range(0, len(align)), len_test)
    selected_alignments = [align[x] for x in selected_numbers]
    SeqIO.write(selected_alignments, f"{OUTPUT_FOLDER_PATH}test/{PF_selected}_{taxon_selected}_{len_test}_{_}.fasta", "fasta")
    _+=1

#val
_ = 0
while _ < n_val:
    #select random PF from val subset
    PF_selected = random.choice(eukaryota_split_nested_list[0])
    #select random .afa for this PF
    alignment_selected = random.choice(list(alignments_df[alignments_df["PF"] == PF_selected]["alignment"]))
    taxon_selected = alignments_df[alignments_df["alignment"] == alignment_selected][["taxon"]].values[0][0]
    #read alignment, subset randomly specified amount of sequences and write them
    align = AlignIO.read(f"{EUKARYOTA_TREES_PATH}{taxon_selected}/Alignments/{alignment_selected}", "fasta")
    selected_numbers = random.sample(range(0, len(align)), len_val)
    selected_alignments = [align[x] for x in selected_numbers]
    SeqIO.write(selected_alignments, f"{OUTPUT_FOLDER_PATH}val/{PF_selected}_{taxon_selected}_{len_val}_{_}.fasta", "fasta")
    _+=1



sh: 1: /home/ruslan_gumerov/genitor_new/final_subsets/eukaryota//train/: Permission denied
sh: 1: /home/ruslan_gumerov/genitor_new/final_subsets/eukaryota//test/: Permission denied
sh: 1: /home/ruslan_gumerov/genitor_new/final_subsets/eukaryota//val/: Permission denied


In [15]:
#Validation

print(f"Amount of alignments in train: {len(os.listdir(train_path))}")
print("Number of sequences in the train alignment:", os.popen(f"grep -o '>' {train_path}{os.listdir(train_path)[0]} | wc -l").read())

print(f"Amount of alignments in test: {len(os.listdir(test_path))}")
print("Number of sequences in the train alignment:", os.popen(f"grep -o '>' {test_path}{os.listdir(test_path)[0]} | wc -l").read())

print(f"Amount of alignments in val: {len(os.listdir(val_path))}")
print("Number of sequences in the train alignment:", os.popen(f"grep -o '>' {val_path}{os.listdir(val_path)[0]} | wc -l").read())

Amount of alignments in train: 1000
Number of sequences in the train alignment: 12

Amount of alignments in test: 300
Number of sequences in the train alignment: 30

Amount of alignments in val: 300
Number of sequences in the train alignment: 20



## Results stored in /home/ruslan_gumerov/genitor_new/final_subsets/eukaryota/

# 4. Check average RF distance for test 

In [ ]:
#S.A.S. parameters – 900 alignments, 30 sequences, 0.652 average


#path to initial matrices, normalized by Ala-Ala = 4
#path to pq output trees
#path to input alignments
#path to consensus trees
PATH_TO_MATRICES = "/home/ruslan_gumerov/genitor_new/matrices/initial_normalized/"
PATH_TO_PQ_TREES = "/home/ruslan_gumerov/genitor_new/pq_trees/prokaryota/test/"
PATH_TO_ALIGNMENTS = "/home/ruslan_gumerov/genitor_new/alignments/prokaryota/test/"
OUT_TREES_PATH = "/home/ruslan_gumerov/genitor_new/pq_trees/prokaryota/test/"

final_rf = []
for matrice in os.listdir(PATH_TO_MATRICES):
    #list to store rf values
    average_rf_list = []
    for alignment in os.listdir(PATH_TO_ALIGNMENTS):
        matrice_file_path = f"{PATH_TO_MATRICES}{matrice}"
        alignment_file_path = f"{PATH_TO_ALIGNMENTS}{alignment}"
        consensus_file_path = f"{PROKARYOTA_TREES_PATH}{alignment.split('_')[1]}/consensus/ncbi-consense.tre"
        #run pq for matrice_file and alignment_file, write results to out_path
        test = os.popen(f"pq -alignment {alignment_file_path} -pwm {matrice_file_path} -grType one -nniType none -out {OUT_TREES_PATH}pq_out.tre").read()
        #run rf for out_path tree and consensus file, add results to list
        rf = float(os.popen(f"rf_dist {OUT_TREES_PATH}pq_out.tre {consensus_file_path}").read().strip())
        average_rf_list.append(rf)
        #rm out pq_out tree for this particular alignment/matrice pair
        os.popen(f"rm {OUT_TREES_PATH}pq_out.tre")
    print(f"{matrice} average: {np.mean(average_rf_list)}")
    final_rf.append([matrice, np.mean(average_rf_list)])

## Average RF distance ≈ 0.65  confirmed

In [33]:
for i in final_rf:
    print(i[0], i[1])

blosumG_20-20_35_0.txt 0.6542003333333334
blosumG_20-20_50_0.txt 0.65062
blosumG_20-20_60_0.txt 0.6512373333333333
blosumG_20-20_30_0.txt 0.6570416666666667
blosumG_20-20_75_0.txt 0.6506196666666667
blosumG_20-20_85_0.txt 0.65247
blosumG_20-20_62_0.txt 0.648892
blosumG_20-20_65_0.txt 0.6542006666666667
blosumG_20-20_55_0.txt 0.6523479999999999
blosumG_20-20_80_0.txt 0.6490126666666667
blosumG_20-20_45_0.txt 0.6472876666666666
blosumG_20-20_40_0.txt 0.6525943333333334
blosumG_20-20_70_0.txt 0.6509893333333334
blosumG_20-20_90_0.txt 0.650496
blosumG_20-20_100_0.txt 0.6453116666666667
blosumG_20-20_95_0.txt 0.6524733333333333


In [6]:
PATH_TO_SUBSET = "/home/ruslan_gumerov/genitor_new/matrices/200matrices_20mut/"

for file in os.listdir(PATH_TO_SUBSET):
    f = open(f"{PATH_TO_SUBSET}{file}", 'r')
    print(f.read())

	C	F	Y	W	M	L	I	V	G	P	A	T	S	N	H	Q	E	D	R	K	
C	13	-4	-4	-3	-4	-2	0	-2	-4	-5	-2	1	-3	0	-4	-4	-1	-3	-2	0	
F	-4	6	3	-2	-2	1	-1	-1	-4	-6	0	-2	2	-2	0	-4	-2	-2	-3	-1	
Y	-4	3	7	1	0	0	2	-1	-3	-3	-1	0	0	-2	0	1	-1	-2	1	-1	
W	-3	-2	1	12	0	0	0	-3	1	-5	0	-1	-3	-3	-4	1	1	-3	0	-1	
M	-4	-2	0	0	5	2	0	0	-4	-3	1	-1	-1	0	1	2	-1	-2	0	0	
L	-2	1	0	0	2	4	0	2	-4	-2	-2	-1	-2	-1	-1	-1	-1	-1	0	-1	
I	0	-1	2	0	0	0	6	3	-2	-3	0	1	-2	1	-2	-4	-4	-3	-3	-2	
V	-2	-1	-1	-3	0	2	3	4	-1	-3	-1	1	0	-4	-3	-3	0	-2	1	-2	
G	-4	-4	-3	1	-4	-4	-2	-1	6	-3	1	0	1	0	-1	-3	-5	0	1	2	
P	-5	-6	-3	-5	-3	-2	-3	-3	-3	9	-2	2	-1	-1	1	1	0	-2	-1	0	
A	-2	0	-1	0	1	-2	0	-1	1	-2	4	-2	2	0	-1	1	1	0	0	-1	
T	1	-2	0	-1	-1	-1	1	1	0	2	-2	5	2	-1	-1	-1	-1	0	0	1	
S	-3	2	0	-3	-1	-2	-2	0	1	-1	2	2	3	-1	1	-1	-1	1	-2	-1	
N	0	-2	-2	-3	0	-1	1	-4	0	-1	0	-1	-1	4	1	-3	-1	2	-1	1	
H	-4	0	0	-4	1	-1	-2	-3	-1	1	-1	-1	1	1	10	1	0	0	-1	-1	
Q	-4	-4	1	1	2	-1	-4	-3	-3	1	1	-1	-1	-3	1	5	3	-1	1	0	
E	-1	-2	-1	1	-1	-1	-4	0	-5	0	1	-1	-1	-1	0	3	5	1	-1	1	
D	-3	-2	-2	-3	-2	-1	-3	-2	0	-2	0	0	1	2	

# 5. Create initial subset of matrices and print them to stdout

In [16]:
PATH_TO_INITIAL_MATRICES = "/home/ruslan_gumerov/genitor_new/matrices/initial_normalized/"
NUMBER_OF_MATRICES = 300
#keep NUMBER_OF_MUTATIONS = 20 for initial subset (strong mutations for initial subset)
NUMBER_OF_MUTATIONS = 20
PATH_TO_SUBSET = f"/home/ruslan_gumerov/genitor_new/matrices/rank_300_10/"

def matrices_subset_creator(PATH_TO_INITIAL_MATRICES: str, PATH_TO_SUBSET: str, NUMBER_OF_MATRICES: int, NUMBER_OF_MUTATIONS: int):
    #create folder with matrices
    os.system(f"mkdir {PATH_TO_SUBSET}")
    _ = 0
    while _ < NUMBER_OF_MATRICES:
        initial_matrice = random.choice(os.listdir(PATH_TO_INITIAL_MATRICES))
        os.system(f"/home/ruslan_gumerov/genitor/genitor -mutate {PATH_TO_INITIAL_MATRICES}{initial_matrice} {PATH_TO_SUBSET}matrice{_}.txt 1 {NUMBER_OF_MUTATIONS} >/dev/null 2>&1")
        print(f"Command: /home/ruslan_gumerov/genitor/genitor -mutate {PATH_TO_INITIAL_MATRICES}{initial_matrice} {PATH_TO_SUBSET}matrice{_}.txt 1 {NUMBER_OF_MUTATIONS} >/dev/null 2>&1")
        print("file:")
        with open(f"{PATH_TO_SUBSET}matrice{_}.txt") as f:
            for line in f:
                print(line)
        _ += 1
    return 0

matrices_subset_creator(PATH_TO_INITIAL_MATRICES, PATH_TO_SUBSET, NUMBER_OF_MATRICES, NUMBER_OF_MUTATIONS)

Command: /home/ruslan_gumerov/genitor/genitor -mutate /home/ruslan_gumerov/genitor_new/matrices/initial_normalized/blosumG_20-20_70_0.txt /home/ruslan_gumerov/genitor_new/matrices/rank_300_10/matrice0.txt 1 20 >/dev/null 2>&1
file:
	C	F	Y	W	M	L	I	V	G	P	A	T	S	N	H	Q	E	D	R	K	

C	9	-3	-3	-1	-2	-1	-2	0	-1	-4	0	1	-2	-2	-3	-1	-3	-2	-2	-3	

F	-3	5	3	1	1	2	-1	1	-2	-1	-3	-2	1	-3	0	-3	-1	-3	-2	-5	

Y	-3	3	6	1	-1	0	-1	-2	-3	-3	-1	-1	-3	-2	1	-1	-2	-2	-1	-1	

W	-1	1	1	10	0	-1	-4	-2	-2	-3	-2	-4	-4	-3	-2	-2	-2	-5	-2	-4	

M	-2	1	-1	0	5	3	2	1	-4	-2	-2	-2	-3	-1	-2	-1	-2	-1	0	1	

L	-1	2	0	-1	3	3	0	0	-4	-4	-2	-2	-1	-4	-2	-2	-3	-5	-2	-3	

I	-2	-1	-1	-4	2	0	3	3	-2	-3	-3	3	-4	-3	-4	0	-3	-5	-3	-1	

V	0	1	-2	-2	1	0	3	3	-4	-3	1	-3	2	-1	-3	-2	-3	-4	0	-2	

G	-1	-2	-3	-2	-4	-4	-2	-4	6	-3	0	-2	0	1	-1	-3	-1	-2	-2	-1	

P	-4	-1	-3	-3	-2	-4	-3	-3	-3	7	0	-2	-1	-2	-2	-1	-1	0	-2	-1	

A	0	-3	-1	-2	-2	-2	-3	1	0	0	4	-2	-1	-1	-1	0	1	0	-1	0	

T	1	-2	-1	-4	-2	-2	3	-3	-2	-2	-2	3	0	2	0	1	-1	0	-1	1	

S	-2	1	-3	-4	-3	-1	-4	2	0	-1	-1

Command: /home/ruslan_gumerov/genitor/genitor -mutate /home/ruslan_gumerov/genitor_new/matrices/initial_normalized/blosumG_20-20_62_0.txt /home/ruslan_gumerov/genitor_new/matrices/rank_300_10/matrice11.txt 1 20 >/dev/null 2>&1
file:
	C	F	Y	W	M	L	I	V	G	P	A	T	S	N	H	Q	E	D	R	K	

C	8	-3	-2	-1	-1	-1	-1	0	-3	-2	0	1	0	-3	-2	-2	-3	-3	-3	-4	

F	-3	4	2	-3	-1	0	0	0	-2	-5	1	-3	-3	-3	0	-4	-2	-4	-2	-4	

Y	-2	2	6	3	0	-1	0	-2	-4	-2	0	-1	-1	-1	2	-1	-2	-6	-1	0	

W	-1	-3	3	10	-3	0	-1	-2	-1	-3	1	-2	0	-3	-1	0	-2	-3	-1	-2	

M	-1	-1	0	-3	6	2	1	0	-2	-2	3	1	0	-4	0	2	-1	-2	-2	-5	

L	-1	0	-1	0	2	3	0	-1	-2	-2	-2	0	-2	-4	-2	-2	-2	-3	-2	-3	

I	-1	0	0	-1	1	0	4	3	-2	-1	0	0	-3	-2	-1	-2	-4	-4	-2	-1	

V	0	0	-2	-2	0	-1	3	3	-4	-2	2	1	0	-1	-2	-3	-1	-2	-2	-3	

G	-3	-2	-4	-1	-2	-2	-2	-4	5	-3	1	-1	0	0	-3	-1	-1	-1	-3	2	

P	-2	-5	-2	-3	-2	-2	-1	-2	-3	7	0	-2	0	-3	-1	-1	-1	0	-2	-2	

A	0	1	0	1	3	-2	0	2	1	0	4	1	3	-1	-3	2	-2	1	-2	1	

T	1	-3	-1	-2	1	0	0	1	-1	-2	1	3	0	0	0	0	0	-2	-2	1	

S	0	-3	-1	0	0	-2	-3	0	0	0	3	0	3	1	1	0	0	2	0	0	

N	

Command: /home/ruslan_gumerov/genitor/genitor -mutate /home/ruslan_gumerov/genitor_new/matrices/initial_normalized/blosumG_20-20_85_0.txt /home/ruslan_gumerov/genitor_new/matrices/rank_300_10/matrice26.txt 1 20 >/dev/null 2>&1
file:
	C	F	Y	W	M	L	I	V	G	P	A	T	S	N	H	Q	E	D	R	K	

C	7	0	-1	-3	-3	0	-2	0	-4	-3	3	2	-1	-1	-1	-3	-4	-4	-3	-3	

F	0	5	3	1	1	0	1	-1	-2	-3	-4	-2	-2	-5	1	-3	-4	-2	-4	-4	

Y	-1	3	6	1	-1	-2	-1	-3	-4	-3	-3	-3	-3	-3	2	1	-2	-3	-3	-2	

W	-3	1	1	9	-1	-4	0	-1	0	-3	-1	-1	-5	-5	-3	-3	-2	-5	-4	-3	

M	-3	1	-1	-1	4	2	2	2	-4	-2	1	0	-3	-4	-5	0	-3	-2	0	-1	

L	0	0	-2	-4	2	3	2	0	-5	-1	-4	-1	-1	-2	-2	0	-4	-3	-3	-3	

I	-2	1	-1	0	2	2	4	1	-3	-3	0	-1	-1	-2	-1	-3	-3	-4	-2	-2	

V	0	-1	-3	-1	2	0	1	3	-3	-1	0	0	-1	-4	-5	-1	-2	-4	-2	-2	

G	-4	-2	-4	0	-4	-5	-3	-3	4	-3	0	-2	2	0	-1	-2	-2	-2	0	-2	

P	-3	-3	-3	-3	-2	-1	-3	-1	-3	6	2	-1	0	-2	-2	1	-1	-2	-2	-1	

A	3	-4	-3	-1	1	-4	0	0	0	2	4	2	-1	-3	-1	3	-2	-3	0	1	

T	2	-2	-3	-1	0	-1	-1	0	-2	-1	2	3	2	2	0	1	0	-1	-1	0	

S	-1	-2	-3	-5	-3	-1	-1	-1	2	0	-1	2	4	-1	-2

Command: /home/ruslan_gumerov/genitor/genitor -mutate /home/ruslan_gumerov/genitor_new/matrices/initial_normalized/blosumG_20-20_30_0.txt /home/ruslan_gumerov/genitor_new/matrices/rank_300_10/matrice40.txt 1 20 >/dev/null 2>&1
file:
	C	F	Y	W	M	L	I	V	G	P	A	T	S	N	H	Q	E	D	R	K	

C	17	-3	-5	0	-4	0	-1	0	-3	-3	-1	-1	-2	-1	-4	-2	0	-2	-3	-3	

F	-3	10	3	1	0	3	0	-1	-4	-4	1	-2	-2	0	-1	-3	-2	-4	0	-1	

Y	-5	3	9	3	-3	3	3	1	-2	-2	-4	-1	-2	-4	0	-1	-3	0	-1	-1	

W	0	1	3	19	-2	-4	-3	-3	0	-2	-5	-5	-3	-6	-4	-2	0	-5	0	0	

M	-4	0	-3	-2	7	0	-3	3	-1	-4	0	2	-3	-2	3	2	0	0	-1	1	

L	0	3	3	-4	0	3	3	1	-2	1	-5	-2	-2	1	3	2	1	-2	-2	-1	

I	-1	0	3	-3	-3	3	6	3	1	-2	-2	0	-1	0	-3	-1	-4	-3	-1	-3	

V	0	-1	1	-3	3	1	3	3	-2	-3	0	1	-2	-1	-4	-4	-1	1	-1	-2	

G	-3	-4	-2	0	-1	-2	1	-2	8	0	-1	-2	-1	0	-4	-2	-2	-1	-2	0	

P	-3	-4	-2	-2	-4	1	-2	-3	0	11	-2	1	-2	-3	1	0	0	1	0	0	

A	-1	1	-4	-5	0	-5	-2	0	-1	-2	4	1	0	0	-3	0	1	-1	3	-1	

T	-1	-2	-1	-5	2	-2	0	1	-2	1	1	6	1	1	-2	1	-2	2	-2	1	

S	-2	-2	-2	-3	-3	-2	-1	-2	-1	-2	0	1	3	-1	0	-1	-1	0	0	1	

N	

Command: /home/ruslan_gumerov/genitor/genitor -mutate /home/ruslan_gumerov/genitor_new/matrices/initial_normalized/blosumG_20-20_45_0.txt /home/ruslan_gumerov/genitor_new/matrices/rank_300_10/matrice53.txt 1 20 >/dev/null 2>&1
file:
	C	F	Y	W	M	L	I	V	G	P	A	T	S	N	H	Q	E	D	R	K	

C	9	-2	-1	-4	-1	1	-1	-1	-3	-3	1	0	1	0	0	-1	-1	-1	-2	-3	

F	-2	5	1	3	-2	-1	1	2	-3	-2	-4	0	-1	-2	-4	-4	-2	-3	-1	-1	

Y	-1	1	5	3	2	-1	-2	0	-3	0	1	-2	-1	1	-1	0	-3	-1	-1	0	

W	-4	3	3	13	-3	-2	-2	-3	-1	0	-1	-1	-2	-4	-4	0	1	-2	-2	-2	

M	-1	-2	2	-3	6	1	2	0	-1	-1	0	-1	1	-1	-1	1	-1	-2	-1	0	

L	1	-1	-1	-2	1	5	1	0	-2	-4	-1	1	0	1	-2	-1	-2	-3	-1	-2	

I	-1	1	-2	-2	2	1	5	0	-2	-1	-2	-1	-3	-4	0	-1	-1	-3	-2	-2	

V	-1	2	0	-3	0	0	0	3	-2	-2	0	0	0	0	-2	-2	0	-1	-3	-2	

G	-3	-3	-3	-1	-1	-2	-2	-2	6	-2	0	-1	0	1	-3	-1	-1	-1	-2	-1	

P	-3	-2	0	0	-1	-4	-1	-2	-2	8	0	2	-2	0	1	-1	0	-3	-3	1	

A	1	-4	1	-1	0	-1	-2	0	0	0	4	0	-1	0	1	1	2	-1	-1	-2	

T	0	0	-2	-1	-1	1	-1	0	-1	2	0	4	1	1	-1	0	0	0	-1	0	

S	1	-1	-1	-2	1	0	-3	0	0	-2	-1	1	3	-1	3	1	0	-1	-1	0	

N	0

...

Command: /home/ruslan_gumerov/genitor/genitor -mutate /home/ruslan_gumerov/genitor_new/matrices/initial_normalized/blosumG_20-20_62_0.txt /home/ruslan_gumerov/genitor_new/matrices/rank_300_10/matrice260.txt 1 20 >/dev/null 2>&1
file:
	C	F	Y	W	M	L	I	V	G	P	A	T	S	N	H	Q	E	D	R	K	

C	8	1	-1	0	0	-1	0	1	-3	-4	-1	0	1	-1	-4	-2	-4	-4	-2	-4	

F	1	6	3	1	0	0	2	3	-5	-5	-3	-3	0	-2	0	-3	-1	-2	0	-4	

Y	-1	3	5	3	0	-1	0	-2	-4	-4	-1	0	-2	-3	2	-3	-1	-3	-1	0	

W	0	1	3	11	0	-2	0	-3	-1	0	-3	-2	-2	-3	-1	0	-1	-3	-2	-1	

M	0	0	0	0	4	3	3	1	-3	-1	-1	0	0	-1	1	-1	-3	-4	-2	0	

L	-1	0	-1	-2	3	3	0	2	-4	-4	-3	-3	-2	-4	-2	-3	-2	-3	-2	-1	

I	0	2	0	0	3	0	3	1	-1	-7	-1	1	0	-5	-2	-2	-2	-5	-2	-1	

V	1	3	-2	-3	1	2	1	3	-3	-3	0	0	-1	-2	-3	-2	-2	-3	-2	-2	

G	-3	-5	-4	-1	-3	-4	-1	-3	5	0	0	-1	0	0	0	-1	-2	-1	-2	-2	

P	-4	-5	-4	0	-1	-4	-7	-3	0	7	-1	-4	3	-2	0	-1	-2	-2	-1	1	

A	-1	-3	-1	-3	-1	-3	-1	0	0	-1	4	0	3	-1	-1	0	0	-1	-1	-1	

T	0	-3	0	-2	0	-3	1	0	-1	-4	0	4	-1	0	-1	0	0	-1	-1	0	

S	1	0	-2	-2	0	-2	0	-1	0	3	3	-1	3	-1	-3	0	0	1	2	-1	

N	

Command: /home/ruslan_gumerov/genitor/genitor -mutate /home/ruslan_gumerov/genitor_new/matrices/initial_normalized/blosumG_20-20_95_0.txt /home/ruslan_gumerov/genitor_new/matrices/rank_300_10/matrice271.txt 1 20 >/dev/null 2>&1
file:
	C	F	Y	W	M	L	I	V	G	P	A	T	S	N	H	Q	E	D	R	K	

C	6	-3	-2	-3	0	-2	-1	-2	-3	-3	1	-1	-1	-1	-2	-3	-4	-4	-2	-3	

F	-3	5	3	1	2	-1	2	-2	-3	-3	0	-2	-1	-6	0	-1	-3	-1	-3	-3	

Y	-2	3	6	1	-1	-1	0	-4	-4	-2	-3	-2	-2	0	1	-1	-3	0	-1	-2	

W	-3	1	1	8	-3	-2	-1	-4	-3	-5	-2	-3	-3	-3	-1	-1	-3	-3	-2	-3	

M	0	2	-1	-3	4	2	-1	0	-4	-3	1	-2	-2	-2	-2	1	-4	-5	-3	0	

L	-2	-1	-1	-2	2	3	0	-1	-5	-4	-2	-1	-2	-3	-2	-3	-4	-4	-3	-2	

I	-1	2	0	-1	-1	0	5	3	-4	-4	-3	-1	-3	-3	-3	-1	-1	-4	-2	-1	

V	-2	-2	-4	-4	0	-1	3	3	-2	-1	-2	2	-1	-1	-2	-3	-1	-3	1	-3	

G	-3	-3	-4	-3	-4	-5	-4	-2	3	-1	-1	-2	1	-6	-5	0	-2	0	-2	-1	

P	-3	-3	-2	-5	-3	-4	-4	-1	-1	6	0	-2	-1	-4	-2	-2	-2	-4	-1	-2	

A	1	0	-3	-2	1	-2	-3	-2	-1	0	4	-1	-1	-1	-1	0	-1	0	0	0	

T	-1	-2	-2	-3	-2	-1	-1	2	-2	-2	-1	4	1	-2	0	-2	0	-2	-1	-1	

S	-1	-1	-2	-3	-2

Command: /home/ruslan_gumerov/genitor/genitor -mutate /home/ruslan_gumerov/genitor_new/matrices/initial_normalized/blosumG_20-20_80_0.txt /home/ruslan_gumerov/genitor_new/matrices/rank_300_10/matrice286.txt 1 20 >/dev/null 2>&1
file:
	C	F	Y	W	M	L	I	V	G	P	A	T	S	N	H	Q	E	D	R	K	

C	7	-2	-6	-5	-1	-1	-2	0	-3	-4	0	-2	-3	-1	-4	-4	-4	-2	-4	-4	

F	-2	6	1	0	1	1	2	0	0	-5	-1	-2	-5	-3	-1	-3	-3	-2	-3	-2	

Y	-6	1	7	3	1	-2	0	-2	-6	-3	0	0	0	-2	2	-3	1	-4	-2	-1	

W	-5	0	3	8	1	-3	-1	-3	-1	-4	-6	-3	-3	-2	-4	-1	-4	-2	-4	-3	

M	-1	1	1	1	5	1	0	-1	-4	-3	0	0	-1	-2	-2	0	-2	-2	0	-1	

L	-1	1	-2	-3	1	3	2	0	-2	-3	-1	-1	-2	-4	-2	-2	-3	-5	-2	-2	

I	-2	2	0	-1	0	2	6	0	-6	-1	1	0	-5	-3	-2	-1	-1	-4	-5	0	

V	0	0	-2	-3	-1	0	0	3	-7	-2	0	-1	0	-2	-5	-2	-1	-3	-1	0	

G	-3	0	-6	-1	-4	-2	-6	-7	4	-2	-2	-2	2	-2	-3	-3	-2	-1	-2	-1	

P	-4	-5	-3	-4	-3	-3	-1	-2	-2	5	-1	0	-3	-2	-2	-1	-2	-1	-3	-1	

A	0	-1	0	-6	0	-1	1	0	-2	-1	4	0	3	0	0	0	1	-1	-3	-1	

T	-2	-2	0	-3	0	-1	0	-1	-2	0	0	4	2	-1	-1	0	0	-1	-1	0	

S	-3	-5	0	-3	-1	-2	-5	0	2	-3	3	2	3	-1	-

Command: /home/ruslan_gumerov/genitor/genitor -mutate /home/ruslan_gumerov/genitor_new/matrices/initial_normalized/blosumG_20-20_55_0.txt /home/ruslan_gumerov/genitor_new/matrices/rank_300_10/matrice297.txt 1 20 >/dev/null 2>&1
file:
	C	F	Y	W	M	L	I	V	G	P	A	T	S	N	H	Q	E	D	R	K	

C	10	-4	-1	0	1	-3	-2	-1	-2	-2	1	-1	-1	-1	-2	-2	-3	-4	-3	-1	

F	-4	5	3	-1	2	1	1	-3	-1	-2	-5	-4	-4	1	-3	-6	-3	0	-4	-3	

Y	-1	3	6	3	0	2	-1	-1	-2	-2	-2	-1	-2	-4	0	-1	-2	-2	-1	-1	

W	0	-1	3	12	-3	-1	-1	-1	-2	-3	0	-1	-1	-6	-1	0	0	-3	-3	-1	

M	1	2	0	-3	6	-1	2	1	-2	-2	-1	0	-1	-1	-1	-1	-3	-2	0	-1	

L	-3	1	2	-1	-1	3	2	2	-4	-1	-1	-2	-2	-5	-4	0	-2	-4	-3	-4	

I	-2	1	-1	-1	2	2	4	2	-3	-3	-1	-1	-1	-4	-2	-2	-4	-3	-2	-2	

V	-1	-3	-1	-1	1	2	2	3	-3	-2	-1	-1	-1	-1	-3	-2	-4	-2	-1	-2	

G	-2	-1	-2	-2	-2	-4	-3	-3	5	0	0	-2	-1	1	0	-1	-2	-1	-3	-1	

P	-2	-2	-2	-3	-2	-1	-3	-2	0	7	1	-1	0	1	-2	-2	0	-1	-3	0	

A	1	-5	-2	0	-1	-1	-1	-1	0	1	4	-1	0	0	0	1	2	-1	-2	1	

T	-1	-4	-1	-1	0	-2	-1	-1	-2	-1	-1	4	2	0	-1	-1	-1	-1	-1	-1	

S	-1	-4	-2	-1	-1	-2	-1	-1	

0